# SQL Project

In [12]:
import pandas as pd
import sqlite3

path = 'mental_health.sqlite'
database = path

In [13]:
conn = sqlite3.connect(database)

In [19]:
tables = pd.read_sql("""SELECT * FROM sqlite_master WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,Answer,Answer,2,"CREATE TABLE ""Answer"" (\n\t""AnswerText"" VARCHA..."
1,table,Question,Question,1434,"CREATE TABLE ""Question"" (\n\t""questiontext"" VA..."
2,table,Survey,Survey,1438,"CREATE TABLE ""Survey"" (\n\t""SurveyID"" INTEGER ..."


In [21]:
answers = pd.read_sql("""SELECT * FROM ANSWER;""", conn)
answers

,AnswerText,SurveyID,UserID,QuestionID
0,37,2014,1,1
1,44,2014,2,1
2,32,2014,3,1
3,31,2014,4,1
4,31,2014,5,1
...,...,...,...,...
236893,Other,2016,2689,117
236894,Support,2016,2690,117
236895,Back-end Developer,2016,2691,117
236896,DevOps/SysAdmin,2016,2692,117


In [23]:
questions = pd.read_sql("""SELECT * FROM QUESTION;""", conn)
questions 

,questiontext,questionid
0,What is your age?,1
1,What is your gender?,2
2,What country do you live in?,3
3,"If you live in the United States, which state ...",4
4,Are you self-employed?,5
...,...,...
100,Do you think that team members/co-workers woul...,114
101,"If yes, what condition(s) have you been diagno...",115
102,"If maybe, what condition(s) do you believe you...",116
103,Which of the following best describes your wor...,117


In [24]:
survey = pd.read_sql("""SELECT * FROM SURVEY;""", conn)
survey

,SurveyID,Description
0,2014,mental health survey for 2014
1,2016,mental health survey for 2016
2,2017,mental health survey for 2017
3,2018,mental health survey for 2018
4,2019,mental health survey for 2019


In [25]:
query = pd.read_sql("""SELECT q.questionid AS QID,
                     q.questiontext AS QTEXT,
                     a.AnswerText AS The_Answer
                    FROM QUESTION q
                    JOIN ANSWER a ON q.questionid = a.QuestionID
                    WHERE q.questionid = 1""", conn)
query

,QID,QTEXT,The_Answer
0,1,What is your age?,37
1,1,What is your age?,44
2,1,What is your age?,32
3,1,What is your age?,31
4,1,What is your age?,31
...,...,...,...
4213,1,What is your age?,27
4214,1,What is your age?,48
4215,1,What is your age?,50
4216,1,What is your age?,30
